# VGG + LSTM Fall Detection on Videos

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [2]:
%cd '/home/jovyan/work/MED_Fall'

/home/jovyan/work/MED_Fall


In [6]:
projectdir = "/home/jovyan/work/MED_Fall"

## Imports

In [4]:
# Evaluating CNN+RNN models on the dataset

# Imports
import os
import random
from collections import Counter
from datetime import datetime

import numpy as np
import tensorflow as tf
import wandb
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from wandb.keras import WandbCallback

from utils.utility_functions import listdir_nohidden_sorted, safe_mkdir
from vision.vision_utils.cnn_rnn_utils import load_and_split
from vision.vision_utils.dataset import TimeSeriesGenerator

In [10]:
# set random seeds
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

In [7]:
# load one features file to check it has the correct shape
with np.load(
    f"{projectdir}/vision/vision_dataset/vgg_features/Actor_1_Bed_PH.npz"
) as features:
    print(features["arr_0"].shape)

(27720, 512)


In [8]:
# load feature extractor
model = tf.keras.models.load_model(
    f"{projectdir}/vision/models/vgg_feature_extractor.h5"
)

2022-08-24 17:20:35.031347: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-24 17:20:35.031394: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: a2d364c967f3
2022-08-24 17:20:35.031404: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: a2d364c967f3
2022-08-24 17:20:35.031584: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.47.3
2022-08-24 17:20:35.031606: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.129.6
2022-08-24 17:20:35.031615: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 470.129.6 does not match DSO version 510.47.3 -- cannot find working devices in this configuration
2022-08-24 17:20:35.032163: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI

In [11]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""

    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


# define options: all network hyperparameters and dataset spects are defined here.
opt = dotdict(
    {
        "lstm1_units": 64,
        "lstm2_units": 32,
        "dense_units": 32,
        "dropout": 0.4,
        "epochs": 1000,
        "train_actors": [4],
        "val_actors": [1],
        "train_cams": [1, 2, 3, 4, 5, 6, 7],
        "val_cams": [1],
        "seq_len": 20,
        "split_ratio": None,
        "drop_offair": True,
        "undersample": False,
        "batch_size": 40,
        "stride": 1,
        "learning_rate": 1e-4,
        "micro_classes": False,
        "num_features": 512,
    }
)

## Load VGG16 pre-trained features

In [12]:
# load_and_split loads the features and labels of the actors (and respective cameras) specified in the options.
X_train, y_train, X_val, y_val, cams_train, cams_val = load_and_split(
    features_folder=f"{projectdir}/vision/vision_dataset/vgg_features",
    dataset_folder=f"{projectdir}/vision/vision_dataset/ground_truth_new",
    train_actors=opt.train_actors,
    val_actors=opt.val_actors,
    train_cams=opt.train_cams,
    val_cams=opt.val_cams,
    split_ratio=opt.split_ratio,
    drop_offair=opt.drop_offair,
    undersample=opt.undersample,
    micro_classes=opt.micro_classes,
)
print(
    f"\nX_train shape: {X_train.shape}, len y_train: {len(y_train)}, X_val shape: {X_val.shape}, len y_val: {len(y_val)}\n"
)

[STATUS] Load Train Set


  0%|          | 0/6 [00:00<?, ?it/s]

Loading csv datasets:   0%|          | 0/6 [00:00<?, ?it/s]

[STATUS] Load Val Set


  0%|          | 0/16 [00:00<?, ?it/s]

Loading csv datasets:   0%|          | 0/16 [00:00<?, ?it/s]


X_train shape: (39368, 512), len y_train: 39368, X_val shape: (18860, 512), len y_val: 18860



In [15]:
series_gen = TimeSeriesGenerator(opt)
X_train_series, y_train_series, classes = series_gen.get_train_series(
    X_train, y_train, cams_train
)
X_val_series, y_val_series = series_gen.get_val_series(X_val, y_val, cams_val)
print(
    f"\nX_train_series shape: {X_train_series.shape}, len y_train_series: {len(y_train_series)}, X_val_series shape: {X_val_series.shape}, len y_val_series: {len(y_val_series)}\n"
)

100%|██████████| 9891/9891 [00:00<00:00, 71358.06it/s]



X_train_series shape: (20664, 20, 512), len y_train_series: 20664, X_val_series shape: (9891, 20, 512), len y_val_series: 9891



In [16]:
import pandas as pd
pd.Series(y_train_series).value_counts()

0    15140
1     5111
2      413
dtype: int64

In [17]:
print(f"Before resampling - y_train_series shape: {Counter(y_train_series)}")

Before resampling - y_train_series shape: Counter({0: 15140, 1: 5111, 2: 413})


## Resample dataset for balancing classes


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline

undersampling_strategy = "not minority"
# oversampling_strategy = {0: 23890, 2: 6437, 1: 4000}
under = RandomUnderSampler(random_state=2, sampling_strategy=undersampling_strategy)
# over = RandomOverSampler(random_state=2, sampling_strategy=oversampling_strategy)
steps = [("u", under)]
pipeline = Pipeline(steps=steps)
# smt = SMOTETomek(random_state=2)

In [ ]:
print(f"Original X_train_series shape: {X_train_series.shape}")

X_train_series = np.reshape(X_train_series, (-1, 20 * opt.num_features))
print(f"X_train_series reshaped: {X_train_series.shape}")

X_train_series, y_train_series = pipeline.fit_resample(X_train_series, y_train_series)

X_train_series = np.reshape(X_train_series, (-1, 20, opt.num_features))

print(f"After resampling X_train_series shape: {X_train_series.shape}")
print(f"After resampling - y_train_series count: {Counter(y_train_series)}")

### Init WANDB project

In [18]:
# WANDB project initialization
run = wandb.init(
    project="Fall detection CNN + RNN",
    config={
        "model": "LSTM",
        "epochs": opt.epochs,
        "seq_len": opt.seq_len,
        "num_features": opt.num_features,
        "batch_size": opt.batch_size,
        "stride": opt.stride,
        "loss_function": "categorical_crossentropy",
        "architecture": "LSTM",
        "train_actors": opt.train_actors,
        "val_actors": opt.val_actors,
        "train_cams": opt.train_cams,
        "val_cams": opt.val_cams,
        "micro_classes": opt.classes,
        "dropout": opt.dropout,
        "lstm1_units": opt.lstm1_units,
        "lstm2_units": opt.lstm2_units,
        "dense_units": opt.dense_units,
        "learning_rate": opt.learning_rate,
        "split_ratio": opt.split_ratio,
        "drop_offair": opt.drop_offair,
        "undersample": opt.undersample,
    },
)

cfg = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: andreaapi. Use `wandb login --relogin` to force relogin


## LSTM Network

In [22]:
model = Sequential()
model.add(
    LSTM(
        units=cfg.lstm1_units,
        input_shape=(cfg.seq_len, cfg.num_features),
        return_sequences=True,
    )
)
model.add(Dropout(cfg.dropout))
model.add(
    LSTM(
        units=cfg.lstm2_units,
        input_shape=(cfg.seq_len, cfg.num_features),
        return_sequences=False,
    )
)
model.add(Dropout(cfg.dropout))
model.add(Dense(units=cfg.dense_units, activation="relu"))
model.add(Dropout(cfg.dropout))
model.add(Dense(units=np.unique(y_train_series, axis=0).shape[0], activation="softmax"))
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=cfg.loss_function,
    metrics=[
        tf.keras.metrics.CategoricalAccuracy(),
        tf.keras.metrics.Recall(class_id=0),
        tf.keras.metrics.Recall(class_id=1),
        tf.keras.metrics.Recall(class_id=2)
    ],
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 64)            147712    
_________________________________________________________________
dropout (Dropout)            (None, 20, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 9

### Callbacks

In [23]:
dir_path = f"{projectdir}/vision/model_checkpoints/LSTM"

model_checkpoint = ModelCheckpoint(
    filepath=f"{dir_path}/VGG_LSTM_best_epoch.h5",
    monitor="val_loss",
    mode="min",
    save_best_only=True,
    save_weights_only=False,
    initial_value_threshold=0.4,
    verbose=1,
    period=10,
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=30,
    verbose=1,
    mode="min",
    min_delta=1e-5,
    cooldown=1,
    min_lr=1e-6,
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=200,
    verbose=1,
    mode="min",
)

callbacks = [WandbCallback(), reduce_lr, early_stop]

### Train the model

In [24]:
# Train Model
history = model.fit(
    X_train_series,
    y_train_series,
    validation_data=(X_val_series, y_val_series),
    epochs=cfg.epochs,
    callbacks=callbacks,
    batch_size=cfg.batch_size,
)

2022-08-24 17:24:52.391734: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-08-24 17:24:52.391882: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-08-24 17:24:52.405018: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 452 nodes (370), 621 edges (528), time = 5.889ms.
  function_optimizer: function_optimizer did nothing. time = 0.103ms.

Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.


Epoch 1/1000


ValueError: in user code:

    /opt/conda/lib/python3.9/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.9/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.9/site-packages/keras/engine/training.py:835 run_step  **
        outputs = model.train_step(data)
    /opt/conda/lib/python3.9/site-pack

### Evaluate the model

In [54]:
val_logits = model.predict(X_val_series, batch_size=60, verbose=1)

8/8 [==============================] - 0s 5ms/step


In [55]:
# free up memory
# del X_train_series
# del y_train_series
# del X_val_series
# del y_val_series

# gc.collect()

### Log metrics to wandb

In [60]:
y_pred_val_classes = np.argmax(val_logits, axis=1).tolist()

cr = classification_report(y_val_series, y_pred_val_classes)

# wandb.log({"Classification Report": cr})

# wandb.sklearn.plot_roc(y_val_series, val_logits, classes)
# wandb.sklearn.plot_class_proportions(y_train_series, y_val_series, classes)
# wandb.sklearn.plot_precision_recall(y_val_series, val_logits, classes)
# wandb.sklearn.plot_confusion_matrix(y_val_series, y_pred_val_classes, classes)
# wandb.join()

### Classification report

In [61]:
print(cr)

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       463
           1       0.00      0.00      0.00         8

    accuracy                           0.98       471
   macro avg       0.49      0.50      0.49       471
weighted avg       0.97      0.98      0.97       471

